In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from jinja2 import Template
from math import sqrt, pi
from scipy.optimize import brentq
import glob, json
import os
try:
    workdir
except NameError:
    workdir=%pwd
else:
    %cd $workdir

### Faunus simulation input for single RNA 30-mer

In [ ]:
templateRNA = Template("""comment: "polyAde-30 in NaCl solution"
temperature: {{T}}
random: {seed: hardware}
geometry: {type: cuboid, length: {{L}}}
mcloop: {macro: 10, micro: {{micro}}}

atomlist:
    - NU: {sigma: 7, eps: {{eps}}, dp: 2}
    - P: {sigma: 4.6, eps: {{eps}}, dp: 2, q: -1}

moleculelist:
    - rna: 
        structure:
{% for i in range(Nnuc) %}
            - NU: [0,0,{{i*2+1-L/2}}]
            - P:  [0,4,{{i*2+1-L/2}}]{% endfor %} 
        bondlist:
{% for i in range(0,Nnuc*2-2,2) %}
            - harmonic: { index: [{{i}},{{i+1}}], k: 4.8, req: 3.5 }
            - harmonic: { index: [{{i}},{{i+2}}], k: 4.8, req: 5 }{% endfor %}
            - harmonic: { index: [{{Nnuc*2-2}},{{Nnuc*2-1}}], k: 4.8, req: 3.5 }
        excluded_neighbours: 1
            
insertmolecules:
    - rna: {N: 1}

energy:
    - bonded: {}
    - nonbonded_splined:
        default:
            - wca:
                mixing: LB
            - custom:
                cutoff: 1000
                function: lB * q1 * q2 * ( exp(-kappa*r) / r - exp(-kappa*Rc) / Rc )
                constants:
                    lB: {{lB}}
                    kappa: {{kappa}}

moves:
    - transrot: {molecule: rna, repeat: N}
    
analysis:
    - savestate: {file: state.json}
    - savestate: {file: confout.pqr}
    - savestate: {file: confout.gro}
    - xtcfile: {file: traj.xtc, nstep: 1e3}
    - reactioncoordinate: {file: Rg.dat, nstep: 1e3, type: molecule, property: Rg, index: 0}
    - reactioncoordinate: {file: Ree.dat, nstep: 1e3, type: molecule, property: end2end, index: 0}
    - reactioncoordinate: {file: tau.dat, nstep: 1e3, type: molecule, property: helicity, index: 0}
    - systemenergy: {file: energy.dat, nstep: 1e5}
""")

In [ ]:
submit = Template("""#!/bin/bash
#requesting the number of cores needed on exclusive nodes
#SBATCH -N 1
#SBATCH --ntasks-per-node=1
#SBATCH -A lu2022-2-36
#
# job time, change for what your job requires
#SBATCH -t 100:0:0
#
# job name
#SBATCH -J {{name}}
#
# filenames stdout and stderr
#SBATCH -o out
#SBATCH -e err

source /home/gtesei00/.bashrc
module load GCCcore/9.3.0
module load CMake/3.16.4
module load GCC/9.3.0
conda activate faunus

/home/gtesei00/Jun22/faunus/faunus --input inp.json --output outp.json -v0 # --state state.json 
""")

### generate scripts and run simulations

In [ ]:
%cd $workdir
faunus_path = '../../faunus'
dirname = '{:s}_{:.2f}_{:.1f}_{:.2g}'
T = 298
L = 200
R = 8.3145 # J/K/mol
eps = R*T*1e-3 # kJ/mol
fepsw = lambda T : 5321/T+233.76-0.9297*T+0.1417*1e-2*T*T-0.8292*1e-6*T**3
epsw = fepsw(T)
lB = 1.60217662**2/(4*np.pi*8.8541878*epsw*1.38064852*T)*1e7
print(lB)
Nnuc = 30
for cs in [20,100,200,400,600]:
    !mkdir {cs}
    %cd {cs}
    kappa = np.sqrt(8*np.pi*lB*cs*6.022/1e5)/10
    with open('inp.yml', 'w') as input_file:
        input_file.write(templateRNA.render(T=T, L=L, micro=1e6,
                    repeat=10, Nnuc=Nnuc, epsw=epsw, lB=lB, eps=eps, kappa=kappa))
    !{faunus_path}/scripts/yason.py inp.yml > inp.json
    with open('submit.sh', 'w') as submit_file:
        temp = submit.render(name=str(cs))
        submit_file.write(temp)
    #!nohup {faunus_path}/faunus --input inp.json --output outp.json --state state.json &
    #!sbatch submit.sh
    %cd ..